In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import pymysql
import os
import time
import datetime

In [3]:
db_dsml = pymysql.connect(
    host = 'localhost', 
    port = 3306, 
    user = 'stock_user', 
    passwd = 'bigdata', 
    db = 'refined_stock', 
    charset = 'utf8'
)

cursor = db_dsml.cursor()

# 데이터 생성

거래대금 조건을 만족하는 날짜 선정

In [4]:
dic_code2date = {}
OF = open('code_list.txt', 'r')
for code in tqdm(OF):
    sql_query = '''
                SELECT *
                FROM stock_{}
                WHERE Date
                BETWEEN '2017-12-01' AND '2021-12-31'
                '''.format(code)
    code = code[:-1]
    code
    stock = pd.read_sql(sql = sql_query, con = db_dsml)
    lst_stock = stock.values.tolist()
    
    for row in lst_stock:
        date, close, volume = row[0].date().strftime('%Y%m%d'), row[4], row[5]
        trading_value = close * volume
        if trading_value >= 10000000000:   # 거래대금 임계값 : 100억
            if code not in dic_code2date.keys():
                dic_code2date[code] = []
                dic_code2date[code].append(date)
            else:
                dic_code2date[code].append(date)

1561it [02:19, 11.20it/s]


## train 데이터 생성

설명변수: 거래대금 조건을 만족한 날짜 기준으로 D-9 ~D-0 open, high, low, close, trading_value (10일치)

종속변수: D+1 change 2% 상승 여부

In [14]:
OF = open('train.txt', 'w')
for code in tqdm(dic_code2date.keys()):
    sql_query = '''
                SELECT *
                FROM stock_{}
                WHERE Date
                BETWEEN '2018-01-01' AND '2020-12-31'
                '''.format(code)
    stock = pd.read_sql(sql = sql_query, con = db_dsml)
    stock=stock[['Date','Open','High','Low','Close','Volume','Change']] # Date, Open, High, Low, Close, Volume, Change 값만 슬라이싱
    lst_stock = stock.values.tolist()      
    
    for i,row in enumerate(lst_stock):
        #예외 처리 
        if (i < 9) or (i >= len(lst_stock)-1): 
            continue
        
        date = row[0].strftime('%Y%m%d')
        
        if date not in dic_code2date[code]:
            continue
  
        sub_stock = lst_stock[i-9:i+1] # D-9 ~ D-0일
        result=[]
        
        for row2 in sub_stock:
            open_, high, low, close, volume = row2[1:6]
            trading_value = close*volume
            result += [open_, high, low, close, trading_value]

        result = ','.join(map(str,result))
        ratio = str(int(lst_stock[i+1][-1]>=0.02)) # 라벨: 종가 2% 상승 여부 (1,0)
    
        OF.write('\t'.join(map(str,[code, date, result, ratio]))+ '\n')
OF.close()

100%|███████████████████████████████████████| 1530/1530 [01:43<00:00, 14.80it/s]


##  test 데이터 생성

설명변수: 거래대금 조건을 만족한 날짜 기준으로 D-9 ~D-0 open, high, low, close, trading_value (10일치)

종속변수: D+1 change 2% 상승 여부

In [17]:
OF = open('test.txt', 'w', encoding = 'utf-8')
for code in tqdm(dic_code2date.keys()):
    sql_query = '''
                SELECT *
                FROM stock_{}
                WHERE Date
                BETWEEN '2021-01-01' AND '2021-06-31'
                '''.format(code)
    stock = pd.read_sql(sql = sql_query, con = db_dsml)
    stock = stock[['Date','Open','High','Low','Close','Volume','Change']] # Date, Open, High, Low, Close, Volume, Change 값만 슬라이싱
    lst_stock = stock.values.tolist()      
    
    for i,row in enumerate(lst_stock):
        #예외 처리 
        if (i < 9) or (i >= len(lst_stock)-1): 
            continue
        
        date = row[0].strftime('%Y%m%d')
        
        if date not in dic_code2date[code]:
            continue
  
        sub_stock = lst_stock[i-9:i+1] # D-9 ~ D-0일
        pred_price=lst_stock[i+1][-1] #다음날 종가 상승율
        
        result=[]
        for row2 in sub_stock:
            open_, high, low, close, volume = row2[1:6]
            trading_value = close*volume
            result += [open_, high, low, close, trading_value]

        result = ','.join(map(str,result))
        ratio = str(int(lst_stock[i+1][-1]>=0.02)) # 라벨: 종가 2% 상승 여부 (1,0)
    
        OF.write('\t'.join(map(str,[code, date, result, ratio]))+ '\n')
OF.close()

100%|███████████████████████████████████████| 1530/1530 [00:25<00:00, 61.00it/s]


# 데이터셋 생성

- train dataset

In [15]:
IF=open("train.txt",'r')
lst_code_date_train=[]
trainX=[]
trainY=[]
for line in IF:
    code, date, x, y = line.strip().split("\t")
    lst_code_date_train.append([code, date])
    trainX.append(list(map(float, x.split(","))))
    trainY.append(int(y))
trainX=np.array(trainX)
trainY=np.array(trainY)

- test dataset

In [21]:
IF=open("test.txt",'r')
lst_code_date_test=[]
testX=[]
testY=[]
for line in IF:
    code, date, x, y = line.strip().split("\t")
    lst_code_date_test.append([code, date])
    testX.append(list(map(float, x.split(","))))
    testY.append(int(y))
testX=np.array(testX)
testY=np.array(testY)

# 모델

In [23]:
from xgboost import XGBClassifier

xgb = XGBClassifier()
xgb.fit(trainX, trainY)
predY_prob = xgb.predict_proba(testX) #다음날 종가 상승 예측 확률적으로 나타내줌
predY = xgb.predict(testX) #다음날 종가 상승 예측을 0,1로 나타내줌

[19:07:31] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


- 모델 평가지표 확인

In [32]:
from sklearn.metrics import accuracy_score, roc_auc_score
acc=accuracy_score(testY, predY)
print('accuracy Score: ', acc) 

roc_score = roc_auc_score(testY, predY)
print('ROC AUC Score: ', roc_score) 

accuracy Score:  0.7828186155472389
ROC AUC Score:  0.5010411878777162


# 수익률 계산

- 주문일지 작성: 예측한 값이 1(상승)일 확률이 높을수록 많은 양 매수
- 확률 임계값과 매수량은 임의로 생성

In [27]:
lst_output=[]
for (code, date), y in zip(lst_code_date_test, predY_prob):
    if y[1] >= 0.9: 
        lst_output.append([code, date, "buy", "all"])  
        lst_output.append([code, date+"n", "sell", "all"])  
    elif y[1] >= 0.7: 
        lst_output.append([code, date, "buy", "r40"])  
        lst_output.append([code, date+"n", "sell", "all"]) 
    elif y[1] >= 0.6: 
        lst_output.append([code, date, "buy", "r20"])  
        lst_output.append([code, date+"n", "sell", "all"])  
    elif y[1] >= 0.5: 
        lst_output.append([code, date, "buy", "r10"])  
        lst_output.append([code, date+"n", "sell", "all"]) 



lst_output.sort(key=lambda x:x[1]) 
OF=open("order.txt", 'w') 
for row in lst_output:
    OF.write("\t".join(map(str, row))+"\n")
OF.close()

- 주문일지(order.txt)를 받아서 수익률을 계산

In [31]:
start_money = 10000000 # 초기 현금 1천만원
money = start_money
dic_code2num ={}  # 보유 종목

IF=open("order.txt",'r')
for i, line in enumerate(IF): #주문 일지를 한 줄 읽어 옴
    code, date, request, amount = line.strip().split("\t")
    sql_query = '''
                SELECT *
                FROM stock_{}
                WHERE Date
                BETWEEN '2021-01-01' AND '2021-06-31'
                '''.format(code)
    stock = pd.read_sql(sql = sql_query, con = db_dsml)
    lst_stock = stock.values.tolist()
            
    if 'n' in date:
        for id,row in enumerate(lst_stock):
            Date=row[0].strftime("%Y%m%d")
            if Date==date.rstrip('n'):
                next_close=lst_stock[id+1][4]
    else:
        for row1 in(lst_stock):
            Date=row1[0].strftime("%Y%m%d")
            if Date==date:
                today_close=row1[4]
            
    if request == 'buy': # buy인 경우
        if amount.startswith('r'):
            request_money = money * float(amount.lstrip("r")) / 100
        elif amount == 'all':
            request_money = money
        elif amount.isdigit():
            request_money = int(amount)
        # elif amount == ~~~~~    ##### 기타 필요한 매수 요청 옵션이 있을 시 작성
        else:
            raise Exception('Not permitted option')
        request_money = min(request_money, money)
        buy_num = int(request_money / today_close)
        money -= buy_num * today_close  # 현재 금액(money)을 실제 매수액을 뺀 만큼 업데이트
        if code not in dic_code2num:
            dic_code2num[code] = 0
        dic_code2num[code] += buy_num # 보유 종목 데이터에 구매 종목(code)를 매수 개수 만큼 증가
    if request == 'sell': # sell인 경우
        if amount == 'all':
            sell_num = dic_code2num[code]
        # elif amount == ~~~~~    ##### 기타 필요한 매도 요청 옵션이 있을 시 작성
        else:
            raise Exception('Not permitted option')            
        money += sell_num * next_close
        dic_code2num[code] -= sell_num
        if dic_code2num[code] == 0:
            del dic_code2num[code]
IF.close()            
            
if dic_code2num != {}: # 매매가 종료되었는데 보유 종목이 있으면
    raise Exception('Not empty stock') 

print("Final earning rate : {} %".format(str((money-start_money) / start_money * 100)))

Final earning rate : 9.70026 %
